# Set up

In [ ]:
!git clone https://github.com/NLP-Reichman/assignment_1.git
!mv assignment_1/data data
!rm assignment_1/ -r

# Introduction
In this assignment you will be creating tools for learning and testing language models. The corpora that you will be working with are lists of tweets in 8 different languages that use the Latin script. The data is provided either formatted as CSV or as JSON, for your convenience. The end goal is to write a set of tools that can detect the language of a given tweet.
The relevant files are under the data folder:

- en.csv (or the equivalent JSON file)
- es.csv (or the equivalent JSON file)
- fr.csv (or the equivalent JSON file)
- in.csv (or the equivalent JSON file)
- it.csv (or the equivalent JSON file)
- nl.csv (or the equivalent JSON file)
- pt.csv (or the equivalent JSON file)
- tl.csv (or the equivalent JSON file)

In [ ]:
import json
# from google.colab import files
import pandas as pd
import numpy as np
import os
import sys
from collections import Counter
import csv


# Implementation

## Part 1
Implement the function *preprocess* that iterates over all the data files and creates a single vocabulary, containing all the tokens in the data. Our token definition is a single UTF-8 encoded character. So, the vocabulary list is a simple Python list of all the characters that you see at least once in the data.

Note - do NOT lowercase the sentences in whi HW.

In [ ]:
SOS = "ה"
EOS = "ס"
UNK = "ל"

def preprocess() -> list[str]:
    '''
    Return a list of characters, representing the shared vocabulary of all languages
    '''
    # Initialize an empty set to store unique tokens
    vocabulary = set()

    # Path to the data folder
    data_folder = 'assignment_1/data/'

    # Iterate over all files in the data folder
    for filename in os.listdir(data_folder):
        # Check if the file is a CSV or JSON file
        if filename.endswith('.csv'):
            with open(os.path.join(data_folder, filename), 'r', encoding='utf-8') as file:
                reader = csv.reader(file)
                # Skip the header row
                next(reader)
                # Read tweet texts and extract tokens
                for row in reader:
                    tweet_text = row[1]  # Assuming tweet text is in the second column
                    vocabulary.update(tweet_text)
    # Convert the set to a sorted list of characters
    # vocabulary.add(UNK)
    vocabulary.add(SOS)
    vocabulary.add(EOS)
    vocabulary_list = sorted(list(vocabulary))
    
    return vocabulary_list

# Get the shared vocabulary of all languages
vocabulary = preprocess()
print(vocabulary)
print(len(vocabulary))

## Part 2
Implement the function *lm* that generates a language model from a textual corpus. The function should return a dictionary (representing a model) where the keys are all the relevant *n*-1 sequences, and the values are dictionaries with the *n*_th tokens and their corresponding probabilities to occur. For example, for a trigram model (tokens are characters), it should look something like:

{ "ab":{"c":0.5, "b":0.25, "d":0.25}, "ca":{"a":0.2, "b":0.7, "d":0.1} }

which means for example that after the sequence "ab", there is a 0.5 chance that "c" will appear, 0.25 for "b" to appear and 0.25 for "d" to appear.

Note - You should think how to add the add_one smoothing information to the dictionary and implement it.

In [ ]:
def lm(lang: str, n: int, smoothed: bool = False) -> dict[str, dict[str, float]]:
    '''
    Return a language model for the given lang and n_gram (n)
    :param lang: the language of the model
    :param n: the n_gram value
    :return: a dictionary where the keys are n_grams and the values are dictionaries
    '''
    df = pd.read_csv('assignment_1/data/' + lang + '.csv')
    # print(df.head())
    text = ''.join(df['tweet_text'].str.cat(sep=''))

    lm = {}

    for i in range(len(text)-n+1):
        ngram = text[i:i+n-1]
        suffix = text[i+n-1]

        if ngram not in lm:
            lm[ngram] = {}
            lm[ngram][suffix] = 1

        else:
            if suffix in lm[ngram]:
                lm[ngram][suffix] += 1
            else:
                lm[ngram][suffix] = 1

    # print(lm)

    for ngram in lm:
        sum_ngram = sum(lm[ngram].values())
        for suffix in lm[ngram]:
            lm[ngram][suffix] = lm[ngram][suffix] / sum_ngram

    # print(lm)

    return lm


ngrams = lm('en', 2)
len(ngrams)
lm_en2 = lm('en', 2)
print(lm_en2)
print(len(lm_en2))
lm_en3 = lm('en', 3)
print(lm_en3)
print(len(lm_en3))
lm_fr = lm('fr', 3)
print(lm_fr)
print(len(lm_fr))
lm_es = lm('es', 3)
print(lm_es)
print(len(lm_es))

## Part 3
Implement the function *eval* that returns the perplexity of a model (dictionary) running over the data file of the given target language.

In [ ]:
def eval(model: dict, target_lang: str) -> float:
  '''
  Return the perplexity value calculated over applying the model on the text file
  of the target_lang language.
  :param model: the language model
  :param target_lang: the target language
  :return: the perplexity value
  '''
  df = pd.read_csv('assignment_1/data/' + target_lang + '.csv')
  # for row in df.iterrows():
  #   print(row)
  sum_perplexity = 0
  for index, row in df.iterrows():
    # print(row['tweet_text'])
    sum_perplexity += perplexity(row['tweet_text'], model)
  
  sum_perplexity /= (len(df) - 1)
  return sum_perplexity

def perplexity(sentence, model):
  n = len(next(iter((model))))
  length = len(sentence)
  perp = 1.0

  for i in range(len(sentence)-n):
    ngram = sentence[i:i+n]
    suffix = sentence[i+n]
    # print(ngram, suffix)

    if ngram in model:
      if suffix in model[ngram]:
        perp *= model[ngram][suffix]
      else: perp *= 1/V
    else: perp *= 1/V
  # print(sentence)
  if perp == 0: perp = sys.float_info.min
  perp = 1/perp
  perp = perp**(1/length)
  return perp


In [ ]:


def eval_log(model: dict, target_lang: str) -> float:
  '''
  Return the perplexity value calculated over applying the model on the text file
  of the target_lang language.
  :param model: the language model
  :param target_lang: the target language
  :return: the perplexity value
  '''
  df = pd.read_csv('assignment_1/data/' + target_lang + '.csv')
  # for row in df.iterrows():
  #   print(row)
  sum_perplexity = 0
  for index, row in df.iterrows():
    # print(row['tweet_text'])
    sum_perplexity += perplexity_log(row['tweet_text'], model)
  
  sum_perplexity /= (len(df) - 1)
  return sum_perplexity


def perplexity_log(sentence, model):
  n = len(next(iter((model))))
  length = len(sentence)
  min_prob = np.log(sys.float_info.min)
  perp = 0.0

  for i in range(len(sentence)-n):
    ngram = sentence[i:i+n]
    suffix = sentence[i+n]
    # print(ngram, suffix)

    if ngram in model:
      if suffix in model[ngram]:
        perp += np.log(model[ngram][suffix])
      else: perp += min_prob
    else: perp += min_prob
  perp = np.exp(perp)
  if length == 0: return 1
  if perp == 0: return 1
  perp = perp**(-1/length)
  return perp

## Part 4
Implement the *match* function that calls *eval* using a specific value of *n* for every possible language pair among the languages we have data for. You should call *eval* for every language pair four times, with each call assign a different value for *n* (1-4). Each language pair is composed of the source language and the target language. Before you make the call, you need to call the *lm* function to create the language model for the source language. Then you can call *eval* with the language model and the target language. The function should return a pandas DataFrame with the following four columns: *source_lang*, *target_lang*, *n*, *perplexity*. The values for the first two columns are the two-letter language codes. The value for *n* is the *n* you use for generating the specific perplexity values which you should store in the forth column.

In [ ]:
def match() -> pd.DataFrame:
  '''
  Return a DataFrame containing one line per every language pair and n_gram.
  Each line will contain the perplexity calculated when applying the language model
  of the source language on the text of the target language.
  :return: a DataFrame containing the perplexity values
  '''

  languages = ['en', 'es', 'fr', 'in', 'it', 'nl', 'pt', 'tl']
  n_grams = [1, 2, 3, 4]
  # languages = languages[:4]
  # n_grams = n_grams[-2:]

  data = []
  for source_lang in languages:
    for n in n_grams:
      model = lm(source_lang, n)
      for target_lang in languages:
        print(source_lang, n, target_lang)
        data.append([source_lang, target_lang, n, eval(model, target_lang)])

  return pd.DataFrame(data, columns=['source', 'target', 'n', 'perplexity'])

# match1 = match()

In [ ]:
def match_log() -> pd.DataFrame:
  '''
  Return a DataFrame containing one line per every language pair and n_gram.
  Each line will contain the perplexity calculated when applying the language model
  of the source language on the text of the target language.
  :return: a DataFrame containing the perplexity values
  '''

  languages = ['en', 'es', 'fr', 'in', 'it', 'nl', 'pt', 'tl']
  n_grams = [1, 2, 3, 4]
  # languages = languages[:4]
  # n_grams = n_grams[-2:]

  data = []
  for source_lang in languages:
    for n in n_grams:
      model = lm(source_lang, n)
      for target_lang in languages:
        print(source_lang, n, target_lang)
        data.append([source_lang, target_lang, n, eval_log(model, target_lang)])

  return pd.DataFrame(data, columns=['source', 'target', 'n', 'perplexity'])

# match1_log = match_log()

In [ ]:
# # match1.to_csv('match1.csv', index=False)
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     print(match1.loc[match1['n'] == 3].sort_values(by='perplexity', ascending=False))


## Part 5
Implement the *generate* function which takes a language code, *n*, the prompt (the starting text), the number of tokens to generate, and *r*, which is the random seed for any randomized action you plan to take in your implementation. The function should start generating tokens, one by one, using the language model of the given source language and *n*. The prompt should be used as a starting point for aligning on the probabilities to be used for generating the next token.

Note - The generation of the next token should be from the LM's distribution.

In [ ]:
def generate(lang: str, n: int, prompt: str, number_of_tokens: int, r: int) -> str:
  '''
  Generate text in the given language using the given parameters.
  :param lang: the language of the model
  :param n: the n_gram value
  :param prompt: the prompt to start the generation
  :param number_of_tokens: the number of tokens to generate
  :param r: the random seed to use
  '''
  model = lm(lang, n)
  length = len(prompt)
  print(prompt)
  for i in range(number_of_tokens):
    # print(i)
    ngram = prompt[-n+1:]
    # print(ngram)
    prompt += random_from_distribution_dict(model[ngram])
    print(prompt)

  return prompt

def random_from_distribution_dict(dct):
    rand_val = np.random.rand()
    total = 0
    for k, v in dct.items():
        total += v
        if rand_val <= total:
            return k
    assert False, 'unreachable'



prompt = 'how are you friend?'
len(generate('en', 5, prompt, 200, 1))

In [ ]:
def test_match():
    df = match()
    return {
        'df_shape': df.shape,
        'en_en_1': df[(df['source'] == 'en') & (df['target'] == 'en') & (df['n'] == 1)]['perplexity'].values[0],
        'tl_tl_1': df[(df['source'] == 'tl') & (df['target'] == 'tl') & (df['n'] == 1)]['perplexity'].values[0],
        'tl_nl_4': df[(df['source'] == 'tl') & (df['target'] == 'nl') & (df['n'] == 4)]['perplexity'].values[0],
        'all' : df
    }
result_test_match = test_match()

In [ ]:
result_test_match

In [ ]:
def test_match_log():
    df = match_log()
    return {
        'df_shape': df.shape,
        'en_en_1': df[(df['source'] == 'en') & (df['target'] == 'en') & (df['n'] == 1)]['perplexity'].values[0],
        'tl_tl_1': df[(df['source'] == 'tl') & (df['target'] == 'tl') & (df['n'] == 1)]['perplexity'].values[0],
        'tl_nl_4': df[(df['source'] == 'tl') & (df['target'] == 'nl') & (df['n'] == 4)]['perplexity'].values[0],
        'all' : df
    }
result_test_match_log = test_match_log()

In [ ]:
result_test_match_log

## Part 6
Play with your generate function, try to generate different texts in different language and various values of *n*. No need to submit anything of that.

# Testing

Copy the content of the **tests.py** file from the repo and paste below. This will create the results.json file and download it to your machine.

In [ ]:
####################
# PLACE TESTS HERE #

# Create tests
def test_preprocess():
    return {
        'vocab_length': len(preprocess()),
    }

def test_lm():
    return {
        'english_2_gram_length': len(lm('en', 2)),
        'english_3_gram_length': len(lm('en', 3)),
        'french_3_gram_length': len(lm('fr', 3)),
        'spanish_3_gram_length': len(lm('es', 3)),
    }

def test_eval():
    return {
        'english_on_english': round(eval(lm('en', 3), 'en'), 2),
        'english_on_french': round(eval(lm('en', 3), 'fr'), 2),
        'english_on_spanish': round(eval(lm('en', 3), 'es'), 2),
    }

def test_match():
    df = match()
    return {
        'df_shape': df.shape,
        'en_en_1': df[(df['source'] == 'en') & (df['target'] == 'en') & (df['n'] == 1)]['perplexity'].values[0],
        'tl_tl_1': df[(df['source'] == 'tl') & (df['target'] == 'tl') & (df['n'] == 1)]['perplexity'].values[0],
        'tl_nl_4': df[(df['source'] == 'tl') & (df['target'] == 'nl') & (df['n'] == 4)]['perplexity'].values[0],
    }

def test_generate():
    return {
        'english_2_gram': generate('en', 2, "I am", 20, 5),
        'english_3_gram': generate('en', 3, "I am", 20, 5),
        'english_4_gram': generate('en', 4, "I Love", 20, 5),
        'spanish_2_gram': generate('es', 2, "Soy", 20, 5),
        'spanish_3_gram': generate('es', 3, "Soy", 20, 5),
        'french_2_gram': generate('fr', 2, "Je suis", 20, 5),
        'french_3_gram': generate('fr', 3, "Je suis", 20, 5),
    }

TESTS = [test_preprocess, test_lm, test_eval, test_match, test_generate]

# Run tests and save results
res = {}
for test in TESTS:
    print(f'Running test: {test.__name__}')
    try:
        cur_res = test()
        res.update({test.__name__: cur_res})
    except Exception as e:
        res.update({test.__name__: repr(e)})


# save the result file as results.json
with open('results.json', 'w') as f:
    json.dump(res, f, indent=2)


# with open('results.json', 'w') as f:
#     json.dump(res, f, indent=2)

# Download the results.json file
# files.download('results.json')

####################

In [ ]:
# Show the local files, results.json should be there now and
# also downloaded to your local machine
!ls -l